In [1]:
import pandas as pd
import pymongo
import os
import json
import datetime as dt

# 1. Connect to Mongo

In [2]:
def read_mongo_config(config_path):
    with open(config_path) as file:
        temp = json.load(file)
    return temp
    

def get_mongo_connection(mongo_config):
    """
    Path to mongo credentials must be set in src/config.py. Returns the Mongo client.
    :return: pymongo.Client
    """
    mongo_url = "mongodb://{}:{}@{}:{}/?authSource={}".format(mongo_config["user"],
                                                              mongo_config["password"],
                                                              mongo_config["host"],
                                                              mongo_config["port"],
                                                              mongo_config["authenticationDatabase"])

    client = pymongo.MongoClient(mongo_url)
    return client

In [3]:
config_path = os.path.join(os.getcwd(), "mongo_config.json")
mongo_config = read_mongo_config(config_path)
mongo_config

{'host': 'localhost',
 'port': '27017',
 'user': 'username',
 'password': 'password',
 'authenticationDatabase': 'auth-database'}

In [4]:
def get_mongo_connection_no_auth(host="localhost", port=27017):
    mongo_url = f"mongodb://{host}:{port}/"
    client = pymongo.MongoClient(mongo_url)
    return client

client = get_mongo_connection_no_auth()

# 2. Write data Data to Mongo

First load data from `clients.json` and `requests.json`

In [5]:
file_path = os.path.join(os.getcwd(), "clients.json")
with open(file_path) as file:
    data = json.load(file)
    df = pd.DataFrame(data)
    df = df.drop(columns=['_id'])
    clients_docs = df.to_dict('records')
clients_docs[0]

{'home_address': '1234 Example Street Chattanooga, TN 37408',
 'first_name': 'First',
 'last_name': 'Last',
 'home_lat': 1,
 'home_lon': -1,
 'wc_type': 1,
 'client_id': 4622,
 'phone': 1231231234,
 'email': 'example@gmail.com'}

In [6]:
file_path = os.path.join(os.getcwd(), "requests.json")
with open(file_path) as file:
    data = json.load(file)
    df = pd.DataFrame(data)
    df = df.drop(columns=['_id'])
    df['requested_pickup_dt'] = df['requested_pickup_dt'].apply(lambda x: dt.datetime.fromisoformat(x['$date'][0:-1]))
    df['pickup_loc'] = df.apply(lambda row: [row['pickup_lon'], row['pickup_lat']], axis=1)
    df['dropoff_loc'] = df.apply(lambda row: [row['dropoff_lon'], row['dropoff_lat']], axis=1)
    requests_docs = df.to_dict('records')
requests_docs[0]

{'pickup_address': 'TestPU',
 'dropoff_address': 'TestDO',
 'pickup_lat': 35.045644,
 'pickup_lon': -85.319982,
 'dropoff_lat': 35.022033,
 'dropoff_lon': -85.241765,
 'am': 0,
 'wc': 1,
 'wc_type': 1,
 'requested_pickup_dt': Timestamp('2021-01-01 09:15:00'),
 'client_id': 4622,
 'booking_id': 1014492,
 'ada': False,
 'pickup_loc': [-85.319982, 35.045644],
 'dropoff_loc': [-85.241765, 35.022033]}

Write data in `clients.json` to the `demo_db` database, `clients` collection

In [7]:
result = client['demo_db']['clients'].insert_many(clients_docs)
result.inserted_ids[0:2]

[ObjectId('6215417e398c70c9054d52e5'), ObjectId('6215417e398c70c9054d52e6')]

Write data in `requests.json` to the `demo_db` database, `requests` collection

In [8]:
result = client['demo_db']['requests'].insert_many(requests_docs)
result.inserted_ids[0:2]

[ObjectId('62154185398c70c9054d552c'), ObjectId('62154185398c70c9054d552d')]

# 3. Example Queries

Get one random document from `clients`

In [9]:
doc = client['demo_db']['clients'].find_one()
doc

{'_id': ObjectId('6215417e398c70c9054d52e5'),
 'home_address': '1234 Example Street Chattanooga, TN 37408',
 'first_name': 'First',
 'last_name': 'Last',
 'home_lat': 1,
 'home_lon': -1,
 'wc_type': 1,
 'client_id': 4622,
 'phone': 1231231234,
 'email': 'example@gmail.com'}

Get all documents from `clients`

In [10]:
docs = list(client['demo_db']['clients'].find())
docs[0:2]

[{'_id': ObjectId('6215417e398c70c9054d52e5'),
  'home_address': '1234 Example Street Chattanooga, TN 37408',
  'first_name': 'First',
  'last_name': 'Last',
  'home_lat': 1,
  'home_lon': -1,
  'wc_type': 1,
  'client_id': 4622,
  'phone': 1231231234,
  'email': 'example@gmail.com'},
 {'_id': ObjectId('6215417e398c70c9054d52e6'),
  'home_address': '928 James Avenue, Chattanooga, Tennessee 37421',
  'first_name': 'Ahmed',
  'last_name': 'Merrill',
  'home_lat': 1,
  'home_lon': -1,
  'wc_type': 1,
  'client_id': 3711,
  'phone': '(262) 414-2190',
  'email': 'Ahmed.Merrill@gmail.com'}]

Convert documents to a pandas dataframe

In [11]:
df = pd.DataFrame(docs)
df.head()

,_id,home_address,first_name,last_name,home_lat,home_lon,wc_type,client_id,phone,email
0,6215417e398c70c9054d52e5,"1234 Example Street Chattanooga, TN 37408",First,Last,1,-1,1,4622,1231231234,example@gmail.com
1,6215417e398c70c9054d52e6,"928 James Avenue, Chattanooga, Tennessee 37421",Ahmed,Merrill,1,-1,1,3711,(262) 414-2190,Ahmed.Merrill@gmail.com
2,6215417e398c70c9054d52e7,"1234 Example Street Chattanooga, TN 37408",First,Last,1,-1,1,6287,1231231234,example@gmail.com
3,6215417e398c70c9054d52e8,"1234 Example Street Chattanooga, TN 37408",First,Last,1,-1,1,4467,1231231234,example@gmail.com
4,6215417e398c70c9054d52e9,"1234 Example Street Chattanooga, TN 37408",First,Last,1,-1,1,6341,1231231234,example@gmail.com


Get all documents from `clients` where `client_id==4622`

In [12]:
query = {"client_id": 4622}
docs = list(client['demo_db']['clients'].find(query))
docs

[{'_id': ObjectId('6215417e398c70c9054d52e5'),
  'home_address': '1234 Example Street Chattanooga, TN 37408',
  'first_name': 'First',
  'last_name': 'Last',
  'home_lat': 1,
  'home_lon': -1,
  'wc_type': 1,
  'client_id': 4622,
  'phone': 1231231234,
  'email': 'example@gmail.com'}]

Range Query - get all documents from `requests` between 1/1/2021 and 1/10/2021

In [13]:
query = {"requested_pickup_dt": 
         {"$gte": dt.datetime(2021, 1, 1), "$lte": dt.datetime(2021, 1, 10)}
        }
docs = pd.DataFrame(list(client['demo_db']['requests'].find(query)))
docs['requested_pickup_dt'].min(), docs['requested_pickup_dt'].max()

(Timestamp('2021-01-01 09:15:00'), Timestamp('2021-01-09 23:30:00'))

# 4. Geospatial Indexing

In [14]:
client['demo_db']['requests'].create_index([("dropoff_loc", pymongo.GEO2D)])

'dropoff_loc_2d'